In [233]:
#Import necessary modules

import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd


In [234]:
#Creating a session

s=Service('D:/Downloads/chromedriver_win32/chromedriver.exe')
driver = webdriver.Chrome(service=s)

In [235]:
#Creating amazon links for any search term

def get_url(search_term):
    template='https://www.amazon.com/s?k={}&ref=nb_sb_noss_1'
    search_term= search_term.replace(' ','+')
    return template.format(search_term)

In [236]:
#Testing get_url

url = get_url('toys')
print(url)
driver.get(url)

https://www.amazon.com/s?k=toys&ref=nb_sb_noss_1


In [237]:
#Grabbing Links for #of Amazon Listings using Search Term

Links = []

def get_links(search_term, list_name, max_links=30):
    url=get_url(search_term)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    results = soup.find_all('a', {'class':'a-link-normal a-text-normal'})

    for i in range (max_links):
        item= results[i]
        href1 = item.get('href')
        if 'https' not in href1:
            url1= 'https://www.amazon.com' + item.get('href')
        else:
            url1= href1
        list_name.append(url1)


In [238]:
get_links(search_term='toys', list_name=Links)
get_links(search_term='gifts', list_name=Links)
get_links(search_term='electronics', list_name=Links)
get_links(search_term='lego set', list_name=Links)

In [239]:
Results = []

def get_dimensions(list_name, csv_name):
    for item in list_name:
        driver.get(item)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        try:
            title = soup.find('span', {'class':'a-size-large product-title-word-break'}).text.strip()
        except AttributeError:
            continue
        try:
            dimensionsxpath= '//*[@id="productDetails_detailBullets_sections1"]/tbody/tr[1]/td'
            dimensions = driver.find_element(By.XPATH, dimensionsxpath).text.strip()
        except NoSuchElementException or AttributeError:
            dimensions = ''
        try:
            weightxpath= '//*[@id="productDetails_detailBullets_sections1"]/tbody/tr[2]/td'
            weight = driver.find_element(By.XPATH, weightxpath).text.strip()
        except NoSuchElementException or AttributeError:
            weight = ''
        result = [title, dimensions, weight]
        csv_name.append(result)
        # time.sleep(2)


In [240]:
get_dimensions(list_name=Links,csv_name=Results)

In [241]:
with open('amazon_dimensions_weight_102621_2.csv','w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Title','Dimensions','Weight'])
    writer.writerows(Results)